# Группировки как фильтр
Считаем метрики для пользователей, которые выставили 100 и более оценок

In [7]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
ratings.groupby('userId').count().head()

,movieId,rating,timestamp
userId,,,
1,20,20,20
2,76,76,76
3,51,51,51
4,204,204,204
5,100,100,100


### Упражнение
Получите список пользователей film_fans_user_ids, которые выставили 100 и более оценок фильмам:
1. Для каждого userId посчитайте количество выставленных оценок. Результат сохраните в датафрейм ratings_grouped
2. Отфильтруйте пользователей в ratings_grouped, которые поставили 100 и более оценок. Результат сохраните в датафрейм ratings_filtered
3. Сохраните идентификаторы пользователей датафрейма ratings_filtered в список film_fans_user_ids:
```python
film_fans_user_ids = ratings_filtered['userId'].tolist()
```
4. Отфильтруйте исходный датафрейм ratings для пользователей из списка film_fans_user_ids:
```python
ratings[ ratings['userId'].isin(...) ]
```

In [39]:
ratings_grouped = ratings.groupby('userId').count().head()
ratings_grouped
ratings_filtered = ratings_grouped[ratings_grouped.rating > 100].reset_index()
film_fans_user_ids = ratings_filtered['userId'].tolist()
ratings[ ratings['userId'].isin(film_fans_user_ids) ].head()

,userId,movieId,rating,timestamp
147,4,10,4.0,949810645
148,4,34,5.0,949919556
149,4,112,5.0,949810582
150,4,141,5.0,949919681
151,4,153,4.0,949811346


# Сводные таблицы
Прям как в экселе

In [50]:
data = pd.read_csv('ml-latest-small/ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [51]:
# какие оценки поставил пользователь №1

data[ data['userId'] == 1 ]['rating'].value_counts()

2.0    7
3.0    4
4.0    3
2.5    3
1.0    2
3.5    1
Name: rating, dtype: int64

In [53]:
# а для всех пользователей сразу? где: индекс - то что будет в строчках. колумс - то что брать за основку в колонки 
# какой параметр основной таблицыб вальюс - что будет считатьб фгг функция - какая функция будет использоваться
#фулл валью - чтобы таблица была красивая. можно делать несколько индексов

data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0).head()


rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0
2,0,2,0,4,0,36,0,23,0,11
3,0,0,0,1,3,18,9,11,4,5
4,0,5,0,5,0,23,0,52,0,119
5,0,0,1,0,3,3,27,42,19,5


In [55]:
#  два индексаю считается каунт для пары значений

ratings.groupby(['userId','rating' ]).count().head()

movieId  timestamp
userId rating                    
1      1.0           2          2
       2.0           7          7
       2.5           3          3
       3.0           4          4
       3.5           1          1

In [58]:
# можно итоги добавить
data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0,
                 margins = True). tail()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,All
userId,,,,,,,,,,,
668,0,2,0,0,0,7,0,3,0,8,20
669,0,0,0,7,0,14,0,12,0,4,37
670,0,2,0,3,0,5,0,10,0,11,31
671,0,1,0,4,4,9,13,57,8,19,115
All,1101,3326,1687,7271,4449,20064,10538,28750,7723,15095,100004


### Упражнение
Какой пользователь выставил больше всех пятерок?

In [62]:
data[ data['rating'] == 5 ]['userId'].value_counts().head(1)

564    408
Name: userId, dtype: int64

In [69]:
df = data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0)
mx = max(df[5])
df[df[5]== mx]

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
564,0,152,0,187,0,414,0,707,0,408


In [71]:
df = data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0)
df[df[5]==max(df[5])]

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
564,0,152,0,187,0,414,0,707,0,408


In [73]:
df = data.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp',
                      aggfunc = 'count', fill_value = 0).sort_values(5, ascending =False).head(1)
df

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
564,0,152,0,187,0,414,0,707,0,408


# Объединение датафреймов

In [22]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

,user_id,source
0,11,ad
1,22,yandex
2,55,email
3,11,google
4,77,ad


In [23]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


In [24]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

,visits
user_id,
11,2
22,1
55,1
77,1


In [25]:
visits.groupby('user_id').count().reset_index()

,user_id,source
0,11,2
1,22,1
2,55,1
3,77,1


In [26]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [27]:
purchases_pivot.reset_index()

category,user_id,Авто,Дача,Спорт
0,11,0,0,2
1,22,1,0,0
2,55,0,1,0
3,99,1,0,0


In [28]:
visits_grouped.join(purchases_pivot)

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [29]:
visits_grouped.join(purchases_pivot, 
                    how='left')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой

In [30]:
visits_grouped.join(purchases_pivot, 
                    how='right')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0,0,2
22,1.0,1,0,0
55,1.0,0,1,0
99,NaN,1,0,0


### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают расходы.

In [8]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [31]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

,id,name
0,43018,Марков Илья
1,48329,Зарицкая Елизавета
2,51043,Благова Дарья
3,74943,Слепова Елена
4,75029,Гордецкий Максим


In [32]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

,id,debit
0,51043,34500
1,48329,12400
2,74943,89044
3,75029,5355
4,43018,19800


In [33]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

,id,credit
0,51043,22990
1,48329,2500
2,74943,69880
3,75029,6000
4,43018,29000


In [48]:
client = client.merge(earnings,  how='left').merge(spending,  how='left')
client[client.debit < client.credit]

,id,name,debit,credit
0,43018,Марков Илья,19800,29000
4,75029,Гордецкий Максим,5355,6000


In [47]:
# если выражение засунть в скобки можно спокойно переносить код. для текста  - тройные кавычки
client = client.merge(earnings,  how='left')
client = client.merge(spending,  how='left')
client[client.debit < client.credit]

,id,name,debit,credit
0,43018,Марков Илья,19800,29000
4,75029,Гордецкий Максим,5355,6000


In [49]:
client = client.merge(earnings,  how='left').merge(spending,  how='left')
client['delta'] = client.debit - client.credit
client[client.delta < 0]

,id,name,debit,credit,delta
0,43018,Марков Илья,19800,29000,-9200
4,75029,Гордецкий Максим,5355,6000,-645


### INNER join
Оставляет строчки, которые есть в обеих таблицах

In [74]:
visits_grouped.join(purchases_pivot, 
                    how='inner')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0,0,2
22,1,1,0,0
55,1,0,1,0


### Outer join
Оставляет все строчки

In [75]:
visits_grouped.join(purchases_pivot, 
                    how='outer')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0.0,0.0,2.0
22,1.0,1.0,0.0,0.0
55,1.0,0.0,1.0,0.0
77,1.0,NaN,NaN,NaN
99,NaN,1.0,0.0,0.0


### Дубликаты при объединении таблиц

In [76]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [77]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


In [78]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [79]:
#  КАК УДАЛЯТЬ ДУБЛИКАТЫ В ИСХОДНОЙ ТАБЛИЦЕ
# subset - по какому параметру идет проврка, keep -  какую по счету строчку дубликатов оставить 
movies.drop_duplicates(subset = ['movieId'], keep = 'first', inplace = True)
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [80]:
ratings.merge(movies, how = 'left', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [81]:
ratings.merge(movies, how = 'right', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


### Оптимизация хранения данных

In [83]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [84]:
len(ratings) == len(joined)

True

In [86]:
joined.to_csv('joined_ratings.csv', index=False)

### Какой жанр имеет самые высокие рейтинги?

In [87]:
import pandas as pd

In [88]:
genres = ['Drama', 'Action', 'Thriller']

In [89]:
import numpy as np

In [90]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [91]:
len(ratings)

100004

In [92]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [93]:
len(movies)

9125

In [94]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [95]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

True

### Считаем рейтинг жанров

In [3]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [4]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [5]:
joined[genres] = joined.apply(genres_ratings, axis=1)
joined.head()

NameError: name 'joined' is not defined

### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base